### Regularization with interaction:

##### Regularization on the 400 best features interaction.  C = 2 ** -6 = 0.015625

Here are the defaults:
```
LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, 
                   class_weight=None, random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', 
                   verbose=0, warm_start=False, n_jobs=1)
```

In [1]:
#### Imports/setup

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', 60)

from timeit import default_timer as timer

# for the pipeline
from sklearn.pipeline import Pipeline
# for the selectors
from sklearn.preprocessing import FunctionTransformer, StandardScaler,  MaxAbsScaler
# for gluing preprocessed text and numbers together
from sklearn.pipeline import FeatureUnion
# for nans in the numeric data
from sklearn.preprocessing import Imputer

# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer

# metrics
from sklearn.metrics import f1_score, accuracy_score, classification_report

# unflattener
import python.flat_to_labels as ftl

#### Set up a train-test split making sure we have all labels in both splits
from python.multilabel import multilabel_train_test_split

from python.dd_mmll import multi_multi_log_loss, BOX_PLOTS_COLUMN_INDICES

from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel

#### Load the data

In [2]:
# Get data
the_data = pd.read_csv('data/TrainingData.csv', index_col=0)

# take a look
the_data.head()

,Function,Use,Sharing,Reporting,Student_Type,Position_Type,Object_Type,Pre_K,Operating_Status,Object_Description,Text_2,SubFund_Description,Job_Title_Description,Text_3,Text_4,Sub_Object_Description,Location_Description,FTE,Function_Description,Facility_or_Department,Position_Extra,Total,Program_Description,Fund_Description,Text_1
134338,Teacher Compensation,Instruction,School Reported,School,NO_LABEL,Teacher,NO_LABEL,NO_LABEL,PreK-12 Operating,NaN,NaN,NaN,Teacher-Elementary,NaN,NaN,NaN,NaN,1.0,NaN,NaN,KINDERGARTEN,50471.810,KINDERGARTEN,General Fund,NaN
206341,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,NO_LABEL,Non-Operating,CONTRACTOR SERVICES,BOND EXPENDITURES,BUILDING FUND,(blank),Regular,NaN,NaN,NaN,NaN,RGN GOB,NaN,UNDESIGNATED,3477.860,BUILDING IMPROVEMENT SERVICES,NaN,BUILDING IMPROVEMENT SERVICES
326408,Teacher Compensation,Instruction,School Reported,School,Unspecified,Teacher,Base Salary/Compensation,Non PreK,PreK-12 Operating,Personal Services - Teachers,NaN,NaN,TCHER 2ND GRADE,NaN,Regular Instruction,NaN,NaN,1.0,NaN,NaN,TEACHER,62237.130,Instruction - Regular,General Purpose School,NaN
364634,Substitute Compensation,Instruction,School Reported,School,Unspecified,Substitute,Benefits,NO_LABEL,PreK-12 Operating,EMPLOYEE BENEFITS,TEACHER SUBS,GENERAL FUND,"Teacher, Short Term Sub",Regular,NaN,NaN,NaN,NaN,UNALLOC BUDGETS/SCHOOLS,NaN,PROFESSIONAL-INSTRUCTIONAL,22.300,GENERAL MIDDLE/JUNIOR HIGH SCH,NaN,REGULAR INSTRUCTION
47683,Substitute Compensation,Instruction,School Reported,School,Unspecified,Teacher,Substitute Compensation,NO_LABEL,PreK-12 Operating,TEACHER COVERAGE FOR TEACHER,TEACHER SUBS,GENERAL FUND,"Teacher, Secondary (High)",Alternative,NaN,NaN,NaN,NaN,NON-PROJECT,NaN,PROFESSIONAL-INSTRUCTIONAL,54.166,GENERAL HIGH SCHOOL EDUCATION,NaN,REGULAR INSTRUCTION


####  Encode the targets as categorical variables

In [3]:
### bind variable LABELS - these are actually the targets and we're going to one-hot encode them...
LABELS = ['Function',  'Use',  'Sharing',  'Reporting',  'Student_Type',  'Position_Type', 
          'Object_Type',  'Pre_K',  'Operating_Status']

### This turns out to be key.  Submission requires the dummy versions of these vars to be in this order.
LABELS.sort()

# Define the lambda function: categorize_label
categorize_label = lambda x: x.astype('category')

# Convert df[LABELS] to a categorical type
the_data[LABELS] = the_data[LABELS].apply(categorize_label, axis=0)

# Print the converted dtypes
print(the_data[LABELS].dtypes)

Function            category
Object_Type         category
Operating_Status    category
Position_Type       category
Pre_K               category
Reporting           category
Sharing             category
Student_Type        category
Use                 category
dtype: object


#### Save the unique labels for each output (category)

In [4]:
# build a dictionary
the_labels = {col : the_data[col].unique().tolist() for col in the_data[LABELS].columns}
# take a look at one entry
the_labels['Use']

['Instruction',
 'NO_LABEL',
 'O&M',
 'Pupil Services & Enrichment',
 'ISPD',
 'Leadership',
 'Business Services',
 'Untracked Budget Set-Aside']

#### Change fraction to suit.
Note: small fractions will have a hard time ensuring labels in both splits.

In [5]:
# downsize it or not
# df = the_data.sample(frac=0.10, random_state=777) # this seed gets a split with enough labels in both sets
df = the_data.sample(frac=1.0, random_state=777)

#### Get targets as set of one-hot encoded columns

In [6]:
# name these columns
NUMERIC_COLUMNS = ['FTE', 'Total']

# Get labels and convert to dummy variables: label_dummies
label_dummies = pd.get_dummies(df[LABELS])

#### Setting up a train-test split  for modeling

#### ======================== repeat regularization with 400 best features interacted ===================================

Some things to note about the default CountVectorizer and HashingVectorizer:
1. All strings are downcased
2. The default setting selects tokens of 2 or more alphanumeric characters (punctuation is completely ignored and always treated as a token separator).  This means single letter or digit tokens are ignored.
3. If the vectorizer is used to transform another input (e.g. test), any tokens not in the original corpus are ignored.

In [7]:
# define combine_text_columns()
def combine_text_columns(df, to_drop=NUMERIC_COLUMNS + LABELS):
    """ converts all text columns in each row of df to single string """
    # Drop non-text columns that are in the df
    to_drop = set(to_drop) & set(df.columns.tolist())
    text_data = df.drop(to_drop, axis=1)  
    # Replace nans with blanks
    text_data.fillna('', inplace=True)    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

In [8]:
# Import FunctionTransformer
from sklearn.preprocessing import FunctionTransformer

# Get the dummy encoding of the labels
dummy_labels = pd.get_dummies(df[LABELS])

# Get the features in the data
NON_LABELS = [c for c in df.columns if c not in LABELS]

# Split into training and test sets
X_train, X_test, y_train, y_test = multilabel_train_test_split(df[NON_LABELS],
                                                               dummy_labels,
                                                               0.2, 
                                                               seed=123)
# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Use all 0s instead of noise: get_numeric_data
get_numeric_data_hack = FunctionTransformer(lambda x: np.zeros(x[NUMERIC_COLUMNS].shape, dtype=np.float), validate=False)

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class IdentityTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, input_array, y=None):
        return self
    
    def transform(self, input_array, y=None):
        return input_array*1

In [10]:
from itertools import combinations
import numpy as np
from scipy import sparse
from sklearn.base import BaseEstimator, TransformerMixin

class JustInteractions(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2, feature_name_separator="_"):
        self.degree = degree
        self.feature_name_separator = feature_name_separator

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not sparse.isspmatrix_csc(X):
            X = sparse.csc_matrix(X)

        if hasattr(X, "columns"):
            self.orig_col_names = X.columns
        else:
            self.orig_col_names = np.array([str(i) for i in range(X.shape[1])])

        spi = self._create_sparse_interactions(X)
        return spi

    def get_feature_names(self):
        return self.feature_names

    def _create_sparse_interactions(self, X):
        out_mat = []
        self.feature_names = self.orig_col_names.tolist()
        ### degree is always 2 so don't worry about it now.  happens once.
        for sub_degree in range(2, self.degree + 1):
            for col_ixs in combinations(range(X.shape[1]), sub_degree):
                # add name for new column
                name = self.feature_name_separator.join(self.orig_col_names[list(col_ixs)])
                self.feature_names.append(name)

                # get column multiplications value
                out = X[:, col_ixs[0]]
                for j in col_ixs[1:]:
                    out = out.multiply(X[:, j])
            
                out_mat.append(out)
        # theory: out_mat is a list of columns; I need to shstack it      
        # print(out_mat)
        # return sparse.hstack([X] + out_mat)
        # return sparse.csc_matrix(np.array(out_mat))
        return sparse.hstack(out_mat)

In [11]:
2 ** -10

0.0009765625

In [12]:
mod_400_2e_neg10 = Pipeline([
    ('text_features', Pipeline([('selector', get_text_data),
                                ('vectorizer', CountVectorizer(ngram_range=(1,2)))])),
    ('union', FeatureUnion(transformer_list = [
        ('passthrough',              IdentityTransformer()),                                 # the word vecs
        ('get_interact',             Pipeline([('get_best', SelectKBest(chi2, 400)),
                                               ('interact', JustInteractions())]))])),        # 16k new features
    ('scale', MaxAbsScaler()),
    ('clf', OneVsRestClassifier(LogisticRegression(C=0.0009765625), n_jobs=-1))
    ])

#### For log loss we need the probabilities, not the predicted labels

In [13]:
start = timer()
# Fit to the training data
mod_400_2e_neg10.fit(X_train, y_train)
end = timer()
print('fit time: {:0.2f} seconds'.format(end - start))

fit time: 1257.03 seconds


In [14]:
# get probas
start = timer()
mod_400_2e_neg10_train_probas = mod_400_2e_neg10.predict_proba(X_train)
mod_400_2e_neg10_test_probas = mod_400_2e_neg10.predict_proba(X_test)
end = timer()
print('Predict.proba time: {:0.2f} seconds'.format(end - start))

Predict.proba time: 130.85 seconds


In [15]:
print('log loss on training set: {:0.4f}'.format(multi_multi_log_loss(mod_400_2e_neg10_train_probas, 
                                                                      y_train.values, BOX_PLOTS_COLUMN_INDICES)))
print('log loss on test set: {:0.4f}'.format(multi_multi_log_loss(mod_400_2e_neg10_test_probas, 
                                                                      y_test.values, BOX_PLOTS_COLUMN_INDICES)))

log loss on training set: 0.3742
log loss on test set: 0.3765


In [16]:
def report_f1(true, pred):
    the_scores = []
    for target in range(len(LABELS)):
        the_score = f1_score(true[:, target], pred[:, target], average='weighted')
        print('F1 score for target {}: {:.3f}'.format(LABELS[target], the_score))
        the_scores.append(the_score)
    print('Average F1 score for all targets : {:.3f}'.format(np.mean(the_scores)))

def report_accuracy(true, pred):
    the_scores = []
    for target in range(len(LABELS)):
        the_score = accuracy_score(true[:, target], pred[:, target])
        print('Accuracy score for target {}: {:.3f}'.format(LABELS[target], the_score))
        the_scores.append(the_score)
    print('Average accuracy score for all targets : {:.3f}'.format(np.mean(the_scores)))

In [17]:
# ftl wants ndarray, not pd.Dataframe
the_ys = ftl.flat_to_labels(y_test.values)

In [18]:
report_f1(the_ys, ftl.flat_to_labels(mod_400_2e_neg10_test_probas))

report_accuracy(the_ys, ftl.flat_to_labels(mod_400_2e_neg10_test_probas))

C:\Users\saus\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 score for target Function: 0.850
F1 score for target Object_Type: 0.934
F1 score for target Operating_Status: 0.962
F1 score for target Position_Type: 0.907
F1 score for target Pre_K: 0.969
F1 score for target Reporting: 0.936
F1 score for target Sharing: 0.902
F1 score for target Student_Type: 0.934
F1 score for target Use: 0.890
Average F1 score for all targets : 0.920
Accuracy score for target Function: 0.862
Accuracy score for target Object_Type: 0.938
Accuracy score for target Operating_Status: 0.964
Accuracy score for target Position_Type: 0.911
Accuracy score for target Pre_K: 0.971
Accuracy score for target Reporting: 0.938
Accuracy score for target Sharing: 0.907
Accuracy score for target Student_Type: 0.936
Accuracy score for target Use: 0.895
Average accuracy score for all targets : 0.925


#### =========================== predict on holdout set ==================================

In [19]:
# Load the holdout data: holdout
### Over here the file is TestData.csv
holdout = pd.read_csv('data/TestData.csv', index_col=0)

C:\Users\saus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
start = timer()
# Generate predictions: predictions
mod_400_2e_neg10_predictions = mod_400_2e_neg10.predict_proba(holdout)
end = timer()
print('predict time: {} seconds'.format(end - start))

predict time: 36.77539502789023 seconds


In [21]:
pred_mod_400_2e_neg10 = pd.DataFrame(columns=pd.get_dummies(df[LABELS], prefix_sep='__').columns, 
                             index=holdout.index,
                             data=mod_400_2e_neg10_predictions)

pred_mod_400_2e_neg10.to_csv('pred_mod_400_2e_neg10.csv')

In [22]:
%ls pred*

 Volume in drive C is Windows
 Volume Serial Number is 98D3-1752

 Directory of C:\Users\saus\Documents\repos\springboard\driven

07/27/2018  08:03 PM       110,836,188 pred_mod_400_003125.csv
07/27/2018  12:24 PM       112,361,340 pred_mod_400_0125.csv
07/27/2018  03:43 PM       113,136,202 pred_mod_400_0250.csv
07/28/2018  11:23 AM       106,762,899 pred_mod_400_2e_neg10.csv
07/27/2018  10:02 PM       110,047,489 pred_mod_400_2e_neg6.csv
07/27/2018  10:54 PM       109,173,255 pred_mod_400_2e_neg7.csv
07/27/2018  11:32 PM       108,352,745 pred_mod_400_2e_neg8.csv
07/28/2018  03:03 AM       107,573,503 pred_mod_400_2e_neg9.csv
07/25/2018  10:34 PM       113,165,315 pred_mod_reg_033.csv
07/26/2018  07:57 PM       111,944,811 pred_mod_reg_033_new.csv
07/25/2018  10:03 PM       113,165,315 pred_mod_reg_third.csv
07/26/2018  08:17 PM       113,165,315 pred_mod_reg_third_new.csv
07/25/2018  06:09 PM       112,963,749 pred_mod4_text.csv
07/20/2018  04:30 PM            24,368 predict_1_1_2.i

#### Scored xxx C=0.125; pretty sure this has room to tweak C.  

#### =============================== End of mod_reg ============================================

***